In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
import matplotlib.pyplot as plt
import pickle
import torch.nn.functional as F
from tqdm.notebook import tqdm

In [ ]:
# DATA_DIR='/kaggle/input/llm-detect-ai-generated-text/'
# df_train_essays = pd.read_csv(DATA_DIR + "train_essays.csv")

In [ ]:
# df_train_essays_ext = pd.read_csv('/kaggle/input/llm-detect-ai-generated-vs-student-generated-text/LLM.csv')
# df_train_essays_ext.rename(columns = {"Label":"generated"}, inplace=True)
# df_train_essays_ext.rename(columns = {"Text":"text"}, inplace=True)
# df_train_essays_ext['generated']=df_train_essays_ext['generated'].replace({'ai':1,'student':0})


# df_train_essays_ext2 = pd.read_csv('/kaggle/input/daigt-v3-train-dataset/train_v3_drcat_01.csv')
# df_train_essays_ext2.rename(columns = {"label":"generated"}, inplace=True)


# df_train_essays_ext4 = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text-deobfuscation/train_from_LLM-Detect_AI-GT_1MNB-3SGD.csv')
# df_train_essays_ext4.rename(columns = {"label":"generated"}, inplace=True)

# df_train_essays_ext5 = pd.read_csv('/kaggle/input/gemini-pro-llm-daigt-dataset/train_essays_v1.csv')
# df_train_essays_ext5.rename(columns = {"label":"generated"}, inplace=True)

# df_train_essays_ext7 = pd.read_csv('/kaggle/input/daigt-one-place-all-data/concatenated.csv')
# df_train_essays_ext7.rename(columns = {"label":"generated"}, inplace=True)

# df_train_essays_final = pd.concat([df_train_essays[["text", "generated"]],df_train_essays_ext2[["text", "generated"]], df_train_essays_ext4[["text", "generated"]],df_train_essays_ext5[["text", "generated"]],df_train_essays_ext7[["text", "generated"]],df_train_essays_ext[["text", "generated"]]])

In [ ]:
# df_train_essays_final=df_train_essays_final.drop_duplicates(subset='text')

In [ ]:
# X=np.array(df_train_essays_final['text'])
# y=np.array(df_train_essays_final['generated'])

In [ ]:
# import matplotlib.pyplot as plt
# plt.hist(df_train_essays_final['generated'])

In [ ]:
# string_indices = np.where(np.array([isinstance(x, str) for x in X]))

# X_strings = X[string_indices]
# y_strings = y[string_indices]

# print( X_strings.shape)
# print( y_strings.shape)

In [ ]:
# indices=np.isnan(y_strings)
# X_=X_strings[~indices]
# y_=y_strings[~indices]


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_,
#                                                     y_,
#                                                     test_size=0.15,
#                                                     random_state=42)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
y_train=np.load('/content/drive/MyDrive/Inputs/y_train.npy')
y_test=np.load('/content/drive/MyDrive/Inputs/y_test.npy')
X_train=np.load('/content/drive/MyDrive/Inputs/X_train.npy',allow_pickle=True)
X_test=np.load('/content/drive/MyDrive/Inputs/X_test.npy',allow_pickle=True)

In [ ]:
X_train.shape

(104777,)

In [ ]:
import tensorflow as tf
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
p=[str(x) for x in X_train[:50000]]

In [ ]:
encoded_inputs = tokenizer(p, padding=True, truncation=True, return_tensors="tf")

In [ ]:
labels = tf.convert_to_tensor(y_train[:50000])
dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": encoded_inputs["input_ids"], "attention_mask": encoded_inputs["attention_mask"]}, labels))

In [ ]:
model = TFElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator", num_labels=2)


tf_model.h5:   0%|          | 0.00/54.5M [00:00<?, ?B/s]

Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [ ]:
batch_size = 32
train_dataset = dataset.shuffle(buffer_size=100).batch(batch_size)

In [ ]:
model.fit(train_dataset, epochs=1)


1563/1563 [==============================] - 1284s 821ms/step - loss: 0.0163 - accuracy: 0.9961


In [ ]:
t=[str(x) for x in X_test]

In [ ]:
encoded_test=tokenizer(t,padding=True,truncation=True, return_tensors="tf")

In [ ]:
test_pred_logits = model.predict({"input_ids": encoded_test["input_ids"], "attention_mask": encoded_test["attention_mask"]})

578/578 [==============================] - 168s 291ms/step


In [ ]:
test_preds = tf.argmax(test_pred_logits.logits, axis=1).numpy()


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(test_preds,y_test)

0.9898869720404521

In [ ]:
# tokenizer.save_pretrained("tokenizer")
# model.save_pretrained("model")

In [ ]:
import pickle
with open('encoded_inputs.pkl','wb') as f:
  pickle.dump(encoded_inputs,f)

In [ ]:
# with open('encoded_test.pkl','wb') as f:
#   pickle.dump(encoded_test,f)

In [ ]:
np.save('X_train.npy',X_train[:50000])